# FBI analysis

In this notebook, we conduct an analysis of **violent offenses** **data** across the United States, focusing on the year **2012**. This year was selected because it provides the **most consistent** dataset across states.

Our goal is to explore trends in violent offenses, normalize the data relative to population sizes, and identify patterns across states. This includes metrics like **total offenses per state** and **offenses per 100,000 inhabitants** to ensure fair comparisons between regions with different population sizes.

In [7]:
import pandas as pd

# Assuming your data is in a CSV file, load it into a DataFrame
data = pd.read_csv("../../data/CLEAN/2012.csv")

# Ensure state prefixes are consistent
data['state_prefix'] = data['state_prefix'].str.strip().str[:2].str.upper()

# Count per state for the same offense
state_offense_count = data.groupby(['state_prefix', 'offense_type']).size().reset_index(name='count')

# Count per offense
offense_count = data['offense_type'].value_counts().reset_index()
offense_count.columns = ['offense_type', 'count']

# Save results
state_offense_count.to_csv("state_offense_count.csv", index=False)
offense_count.to_csv("offense_count.csv", index=False)

# Display the results
print("Counts per state and offense:")
print(state_offense_count)

print("\nCounts per offense:")
print(offense_count)

with pd.ExcelWriter("offense_analysis.xlsx") as writer:
    state_offense_count.to_excel(writer, sheet_name="State_Offense_Count", index=False)
    offense_count.to_excel(writer, sheet_name="Offense_Count", index=False)

print("Results saved in 'offense_analysis.xlsx'")


Counts per state and offense:
    state_prefix                              offense_type  count
0             AL                                     Arson      1
1             AL                          Assault Offenses    825
2             AL              Burglary/Breaking & Entering    391
3             AL                    Counterfeiting/Forgery     55
4             AL  Destruction/Damage/Vandalism of Property    280
..           ...                                       ...    ...
650           WV                     Prostitution Offenses    125
651           WV                                   Robbery    780
652           WV                              Sex Offenses   1246
653           WV                  Stolen Property Offenses    855
654           WV                     Weapon Law Violations    975

[655 rows x 3 columns]

Counts per offense:
                                offense_type    count
0                     Larceny/Theft Offenses  1582037
1                        

In [8]:
# Save in an xlsx
state_offense_count.to_excel('../../data/CLEAN/statecount.xlsx')
offense_count.to_excel('../../data/CLEAN/offensecount.xlsx')

In [34]:
# Dictionary mapping state prefixes to full state names
state_prefix_to_name = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District of Columbia",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}
data = pd.read_excel("../../data/CLEAN/statecount.xlsx")

# Map the state prefixes to full names
data['state_name'] = data['state_prefix'].map(state_prefix_to_name)

# Save the updated df
data.to_excel("../../data/CLEAN/updatedstatecount.xlsx", index=False)


### Selected violent offenses from previous analysis

In [15]:
violent_offenses = ["Larceny/Theft Offenses", "Assault Offenses", "Destruction/Damage/Vandalism of Property",
                    "Burglary/Breaking & Entering", "Sex Offenses", "Robbery", "Arson", "Kidnapping/Abduction", "Homicide Offenses"]

#### Filter offense types

In [22]:
filtered_data = data[data['offense_type'].isin(violent_offenses)]
filtered_data.to_excel("../../data/CLEAN/violentstatecount.xlsx", index=False)
filtered_data

,Unnamed: 0,state_prefix,offense_type,count,state_name
0,0,AL,Arson,1,Alabama
1,1,AL,Assault Offenses,825,Alabama
2,2,AL,Burglary/Breaking & Entering,391,Alabama
4,4,AL,Destruction/Damage/Vandalism of Property,280,Alabama
8,8,AL,Homicide Offenses,2,Alabama
...,...,...,...,...,...
645,645,WV,Homicide Offenses,63,West Virginia
646,646,WV,Kidnapping/Abduction,117,West Virginia
647,647,WV,Larceny/Theft Offenses,28676,West Virginia
651,651,WV,Robbery,780,West Virginia


# Normalization per capita

In order to better visualize the violent offenses, the count of each offenses was normalized using data from [Fact Monster](https://www.factmonster.com/us/fifty-states/state-population-rank-2012).

Normalization was performed to provide better insights through the metric **offenses per 100k capita**.

In [36]:
# Data about population from Fact Monster
population = pd.read_csv('../../data/CLEAN/state_population_data.csv')
population.sample(10)

,State,Population
26,Oregon,3899353
10,New Jersey,8864590
30,Mississippi,2984926
45,South Dakota,833354
5,Pennsylvania,12763536
33,Utah,2855287
7,Georgia,9919945
15,Indiana,6537334
24,Louisiana,4601893
37,West Virginia,1855413


#### Normalization based on population

In [25]:
# Merge violent offense data with population data based on state name
merged_data = pd.merge(
    filtered_data,
    population,
    left_on="state_name",  # Column in violent offense data
    right_on="State",      # Column in population data
    how="left"             # Keep all violent offense data
)

# Calculate normalized offenses per 100,000 people
merged_data["offenses_per_100k"] = (merged_data["count"] / merged_data["Population"]) * 100000

# Drop unnecessary columns
merged_data_cleaned = merged_data.drop(columns=["Unnamed: 0", "State", merged_data.columns[0]], errors="ignore")

# Save the cleaned and normalized data to a new CSV
output_file_path = "../../data/CLEAN/normalized_violent_offenses.csv"
merged_data_cleaned.to_csv(output_file_path, index=False)
merged_data_cleaned.to_excel("../../data/CLEAN/normalized_violent_offenses.xlsx" )


In [29]:
# Merge violent offense data with population data based on state name
merged_data = pd.merge(
    data,
    population,
    left_on="state_name",  # Column in violent offense data
    right_on="State",      # Column in population data
    how="left"             # Keep all violent offense data
)

# Calculate normalized offenses per 100,000 people
merged_data["offenses_per_100k"] = (merged_data["count"] / merged_data["Population"]) * 100000

# Drop unnecessary columns
merged_data_cleaned = merged_data.drop(columns=["Unnamed: 0", "State", merged_data.columns[0]], errors="ignore")

# Save the cleaned and normalized data to a new CSV
output_file_path = "../../data/CLEAN/normalized_offenses.csv"
merged_data_cleaned.to_csv(output_file_path, index=False)
merged_data_cleaned.to_excel("../../data/CLEAN/normalized_offenses.xlsx")